In [1]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

###########################################################################
# USER SETTINGS
###########################################################################

BANDS = np.array([125, 250, 500, 1000, 2000, 4000,
                  6000, 8000, 10000, 15000, 20000, 24000])

RT_TARGET_LOW = -5
RT_TARGET_HIGH = -25

# Room geometry assumptions (change these according to your real room)
ROOM_VOLUME = 50.0     # m^3
TOTAL_AREA = 60.0      # m^2


###########################################################################
# FILTERING AND DECAY
###########################################################################

def bandpass_filter(x, sr, low, high):
    nyq = 0.5 * sr
    low /= nyq
    high /= nyq
    b, a = signal.butter(2, [low, high], btype='band')
    return signal.filtfilt(b, a, x)


def compute_edc(sig):
    rev = np.cumsum(sig[::-1] ** 2)[::-1]
    return 10 * np.log10(rev / np.max(rev))


def estimate_rt(edc, sr):
    t = np.arange(len(edc)) / sr
    valid = (edc < RT_TARGET_LOW) & (edc > RT_TARGET_HIGH)
    if np.sum(valid) < 5:
        return 0.0
    tv = t[valid]
    ev = edc[valid]
    slope, _ = np.polyfit(tv, ev, 1)
    if slope >= 0:
        return 0.0
    return -(60 / slope)


###########################################################################
# PARAMETER ESTIMATORS
###########################################################################

def absorption_from_rt(RT, V=ROOM_VOLUME, S=TOTAL_AREA):
    if RT <= 0:
        return 0.95
    a = 0.161 * V / (RT * S)
    return float(np.clip(a, 0.01, 0.99))


def scattering_from_erl(filtered, sr, split_ms=80):
    split = int(split_ms * 1e-3 * sr)
    energy = filtered ** 2
    early = np.sum(energy[:split])
    late = np.sum(energy[split:])
    if late <= 0:
        return 0.0
    S = 1 - early / (late + 1e-12)
    return float(np.clip(S, 0.0, 1.0))


def damping_from_rt(RT, RT_max):
    if RT_max <= 0:
        return 0.5
    D = 1 - RT / RT_max
    return float(np.clip(D, 0.0, 1.0))


###########################################################################
# MAIN ANALYSIS
###########################################################################

def analyze_rir_npy(npy_path, sample_rate):
    rir = np.load(npy_path, mmap_mode='r')[0, 0]

    # Accept mono or multi-channel .npy
    if rir.ndim > 1:
        rir = rir[:, 0]

    print(f"\nLoaded RIR: {npy_path}  — {len(rir)} samples, {sample_rate} Hz")

    absorptions, scatterings, dampings, rts = [], [], [], []

    # First pass: maximum RT (for damping normalization)
    temp_rts = []
    for fc in BANDS:
        low, high = fc/np.sqrt(2), fc*np.sqrt(2)
        if high > sample_rate / 2:
            temp_rts.append(0)
            continue
        f = bandpass_filter(rir, sample_rate, low, high)
        edc = compute_edc(f)
        temp_rts.append(estimate_rt(edc, sample_rate))

    RT_max = max(temp_rts) if max(temp_rts) > 0 else 1.0
    print(f"\nMax RT for damping normalization: {RT_max:.3f}\n")

    for i, fc in enumerate(BANDS):
        print(f"### BAND {i+1}: {fc} Hz ###")

        low, high = fc/np.sqrt(2), fc*np.sqrt(2)

        # Above Nyquist → skip
        if high > sample_rate / 2:
            print("Skipping (above Nyquist)")
            absorptions.append(0.95)
            scatterings.append(0.5)
            dampings.append(1.0)
            rts.append(0.0)
            continue

        filtered = bandpass_filter(rir, sample_rate, low, high)
        edc = compute_edc(filtered)
        RT = estimate_rt(edc, sample_rate)
        rts.append(RT)

        # Compute parameters
        a = absorption_from_rt(RT)
        s = scattering_from_erl(filtered, sample_rate)
        d = damping_from_rt(RT, RT_max)

        absorptions.append(a)
        scatterings.append(s)
        dampings.append(d)

        print(f"RT60:        {RT:.3f} s")
        print(f"Absorption:  {a:.3f}")
        print(f"Scattering:  {s:.3f}")
        print(f"Damping:     {d:.3f}")
        print()

    ########################################################################
    # PRINT MATERIAL JSON BLOCKS
    ########################################################################

    print("\n====================================================")
    print("     RECOMMENDED MATERIAL PARAMETERS (12 bands)     ")
    print("====================================================\n")

    print("\"absorption\": [")
    for fc, a in zip(BANDS, absorptions):
        print(f"    {fc:.1f}, {a:.3f},")
    print("],\n")

    print("\"scattering\": [")
    for fc, s in zip(BANDS, scatterings):
        print(f"    {fc:.1f}, {s:.3f},")
    print("],\n")

    print("\"damping\": [")
    for fc, d in zip(BANDS, dampings):
        print(f"    {fc:.1f}, {d:.3f},")
    print("]\n")

    return absorptions, scatterings, dampings, rts


###########################################################################
# RUN
###########################################################################

if __name__ == "__main__":
    npy_file = "/home/student/ss4/soundcam-Dataset/TreatedRoom_preprocessed/Empty/deconvolved.npy"   # <--- Replace with your actual file
    sample_rate = 48000    # <--- Replace with your actual RIR sample rate
    analyze_rir_npy(npy_file, sample_rate)



Loaded RIR: /home/student/ss4/soundcam-Dataset/TreatedRoom_preprocessed/Empty/deconvolved.npy  — 667200 samples, 48000 Hz

Max RT for damping normalization: 0.382

### BAND 1: 125 Hz ###
RT60:        0.382 s
Absorption:  0.351
Scattering:  0.000
Damping:     0.000

### BAND 2: 250 Hz ###
RT60:        0.169 s
Absorption:  0.792
Scattering:  0.000
Damping:     0.557

### BAND 3: 500 Hz ###
RT60:        0.115 s
Absorption:  0.990
Scattering:  0.000
Damping:     0.699

### BAND 4: 1000 Hz ###
RT60:        0.063 s
Absorption:  0.990
Scattering:  0.000
Damping:     0.836

### BAND 5: 2000 Hz ###
RT60:        0.063 s
Absorption:  0.990
Scattering:  0.000
Damping:     0.835

### BAND 6: 4000 Hz ###
RT60:        0.055 s
Absorption:  0.990
Scattering:  0.000
Damping:     0.857

### BAND 7: 6000 Hz ###
RT60:        0.063 s
Absorption:  0.990
Scattering:  0.000
Damping:     0.834

### BAND 8: 8000 Hz ###
RT60:        0.054 s
Absorption:  0.990
Scattering:  0.000
Damping:     0.859

### BAND 9: 10